# MLX LangChain Chatbot
In this notebook, we will build a LangChain-based chatbot from end-to-end using MLX as the LLM-hosting service. This notebook is more or less an amalgamation of concepts covered from other sources, so we will tend to gloss over certain things here but will still provide links to those other resources for those who want to dive deeper.

The headings of markdown cells below will largely correlate to the same blog post on Medium. As such, minimal information will be provided beyond the code below.

## Notebook Setup
Throughout this notebook, we will largely be making use of only MLX and LangChain, along with other standand Python functions. In order to make use of these libraries, you will need to install them appropriately. For example, if you use `pip`, you will want to run the following command:

`pip install mlx mlx-lm langchain langchain-community langchain-core`

In [1]:
# Importing the necessary Python libraries
from mlx_lm import load as mlx_load
from mlx_lm import generate as mlx_generate
from langchain_community.llms.mlx_pipeline import MLXPipeline
from langchain_community.chat_models.mlx import ChatMLX

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Setting constant values to represent model name and directory
MODEL_NAME = 'mistralai/Mistral-7B-Instruct-v0.2'
BASE_DIRECTORY = '../models'
MLX_DIRECTORY = f'{BASE_DIRECTORY}/mlx'
mlx_model_directory = f'{MLX_DIRECTORY}/{MODEL_NAME}'

## A Quick Intro to MLX

In [3]:
# Loading the MLX quantized Mistral 7B model from file
mlx_model, mlx_tokenizer = mlx_load(mlx_model_directory)

# Producing the response (completion) with the MLX model
response = mlx_generate(
    model = mlx_model,
    tokenizer = mlx_tokenizer,
    prompt = 'Write me a haiku about Jupyter notebooks.',
    max_tokens = 1000
)

# Printing the response
print(response)



In the flow of code,
Notebook pages gently turn,
Data's story unfolds.
